<a href="https://colab.research.google.com/github/blackboxradiology/interactive_TF/blob/main/FT_interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ipywidgets as wg
import pandas as pd
from PIL import Image
import numpy as np
from IPython import display
import os
from matplotlib import pyplot as plt
import cv2
import requests
from io import BytesIO

In [7]:
np.seterr(divide = 'ignore') 
np.seterr(invalid = 'ignore') 

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [8]:
img_url = 'https://prod-images-static.radiopaedia.org/images/903193/06eb041ff51e064424ba056806564a_gallery.jpg'
original_img = Image.open(requests.get(img_url, stream = True).raw).convert('L').resize((320,320))


In [9]:
def create_circular_mask(h, w, percent, center=None, radius=None):

    if percent < 0.001:
        percent = 0.001
    if center is None:
        center = (int(w/2), int(h/2))
    if radius is None:
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2) * 1/percent

    mask = dist_from_center <= radius
    return mask

In [22]:
#band filter
def band_filter(bottom_frequency, top_frequency):
    f_shift = np.fft.fft2(original_img)
    f_shift = np.fft.fftshift(f_shift)

    background = np.zeros((320,320))
    background[background == 0] = 128

    bg_shift = np.fft.fft2(background)
    bg_shift = np.fft.fftshift(bg_shift)

    original = np.copy(f_shift)

    bottom_frequency = bottom_frequency / 100
    h, w = np.array(original_img).shape[:2]
    mask = create_circular_mask(h, w, bottom_frequency)

    f_shift[mask] = 0

    top_frequency = top_frequency / 100
    mask = create_circular_mask(h, w, top_frequency)

    f_shift[~mask] = 0

    masked_f_shift = f_shift + bg_shift

    fshift_output = np.fft.ifftshift(masked_f_shift)
    output_array = np.uint8(np.real(np.clip(np.fft.ifft2(fshift_output),0,255)))

    output_image = Image.fromarray(np.uint8(np.real(np.clip(np.fft.ifft2(fshift_output),0,255))))
    
    
    #f_shift_top
    img_array = np.log(masked_f_shift.real)
    img_array = np.nan_to_num(img_array,0)
    img_array[img_array < 0.0] = 0
    img_array = (img_array - img_array.min())/(img_array.max() - img_array.min())
    real_filter = Image.fromarray(np.uint8(img_array * 255))
    real_filter
    
    empty_grid = np.zeros((320,320))
    empty_grid[160] = f_shift.real[160]
    fshift_output = np.fft.ifftshift((empty_grid * 50) + bg_shift)
    output_array = np.uint8(np.real(np.clip(np.fft.ifft2(fshift_output),0,255)))

    one_dim_output = Image.fromarray(np.uint8(np.real(np.clip(np.fft.ifft2(fshift_output),0,255))))

    #draw = ImageDraw.Draw(one_dim_output) 
    #font = ImageFont.truetype('Arial_Classic.ttf', 20) 
    #text = "Band Hz \n" +str(bottom_frequency) + ' - ' + str(top_frequency)
    #draw.text((10, 10), text, font = font, align ="center", fill="black") 

    blank_image = Image.new("L", (640, 640))
    blank_image.paste(original_img, (0,0))
    blank_image.paste(output_image, (320,0))
    blank_image.paste(one_dim_output, (0,320))
    blank_image.paste(real_filter, (320,320))
    return blank_image



In [23]:

slider = wg.IntRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=1,
    description='Freq %:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

slideroutput = wg.Output()
display.display(slider, slideroutput)

def on_value_change(change):
    with slideroutput:
        display.clear_output(wait=True)
        display.display(band_filter(slider.value[0],slider.value[1]))


slider.observe(on_value_change, names='value')


IntRangeSlider(value=(0, 100), description='Freq %:')

Output()